# Stock-Dataset-Analysis-by-Spark


In [ ]:
pip install pyspark

     |████████████████████████████████| 212.3MB 68kB/s 
     |████████████████████████████████| 204kB 18.1MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=18be1f5837266bb6e349a3c80273c2523cfbd15eaf48b1bcba1efc90c240687a
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


#### Start a simple Spark Session

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Assignment2").getOrCreate()

#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [ ]:
walmart = spark.read.csv("walmart_stock.csv",inferSchema=True,header=True)

####  Column names

In [ ]:
walmart.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

####  Records are there in the dataframe

In [ ]:
#here i will use count operation 

walmart.count()

1258

####  Schema look like

In [ ]:
#here i will use count operation to  prints the types of columns in the DataFrame
walmart.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



####  First 5 rows.

In [ ]:
 # here i will use show action to print out the first 5 rows
 
 walmart.show(5)


+----------+------------------+---------+---------+------------------+--------+------------------+
|      Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+----------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+----------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows



####  Statistical info about the columns (count, min, max .. etc)

In [ ]:
#here i will use describe() operation to compute summary statistics of numerical columns in the DataFrame
walmart.describe().show()


+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|      Date|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|      1258|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean|      null| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|      null|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|2012-01-03|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|2016-12-30|         90.800003|        90.970001|            89.25|        90.4700

####  New dataframe with a column called HighVolume Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [ ]:
# here i will use withColumn transformation to Create a new dataframe then  use select transformation subsets the columns

new_dataframe = walmart.withColumn(' HighVolume Ratio', walmart['High']/walmart['Volume'])
new_dataframe.select(' HighVolume Ratio').show()

+--------------------+
|    HighVolume Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



#### The Highest Price

In [ ]:
# here i will use orderBy  transformation to sorts the DataFrame based one or more columns
walmart.orderBy(walmart['High'].desc()).select(['Date']).head(1)[0]['Date']

'2015-01-13'

####  The mean of the Close column

In [ ]:
from pyspark.sql.functions import mean, min, max
walmart.select(mean('Close')).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



#### The max and min of the Volume column

In [ ]:
 
walmart.select(min('Volume'),max('Volume')).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



#### Num of  days was the Close lower than 60 dollars

In [ ]:
#here i will used filter transformation to filter out the rows based on 60 dolar condition  
walmart.filter(walmart['Close'] < 60).count()

81

#### The max High per year?


In [ ]:
from pyspark.sql.functions import year
#Here I will Create a new column Year from existing Date column then group a variables and find the max 
walmart.withColumn("Year", year(walmart["Date"])).groupBy('Year').max().select('Year', "max(High)").show()

+----+---------+
|Year|max(High)|
+----+---------+
|2015|90.970001|
|2013|81.370003|
|2014|88.089996|
|2012|77.599998|
|2016|75.190002|
+----+---------+



####The average Close for each Calendar Month


In [ ]:
from pyspark.sql.functions import month

#Here I will Create a new column Month from existing Date column
monthes = walmart.withColumn("Month", month('Date'))
#After i will Group by month , i will  take average of all other columns then  i will Sort by month using orderby function 
(monthes.select('Month', 'Close').groupBy('Month').avg().
                    select('Month','avg(Close)')).orderBy('Month').show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+

